In [5]:
import tensorflow as tf
from tensorflow import keras
from sklearn.datasets import fetch_california_housing
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

def build_model(n_hidden=1, n_neurons=30, learning_rate=0.01, activation="relu", input_shape=[8]):
    model = keras.models.Sequential()
    model.add(keras.layers.InputLayer(input_shape=input_shape))
    for layer in range(n_hidden):
        model.add(keras.layers.Dense(n_neurons, activation=activation))
    # 回归任务，最后一层不需要激活函数（或可使用线性激活，linear，默认就是linear）
    model.add(keras.layers.Dense(1))
    optimizer = keras.optimizers.SGD(learning_rate=learning_rate)
    # 回归任务使用均方误差损失函数
    model.compile(loss="mean_squared_error", optimizer=optimizer, metrics=["mean_squared_error"])
    return model

housing = fetch_california_housing()

X_train_full, X_test, y_train_full, y_test = train_test_split(
    housing.data, housing.target, random_state=42
)

X_train, X_valid, y_train, y_valid = train_test_split(
    X_train_full, y_train_full, random_state=42
)

scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_valid = scaler.transform(X_valid)
X_test = scaler.transform(X_test)

keras_reg = keras.wrappers.scikit_learn.KerasRegressor(build_model)

# 训练模型
keras_reg.fit(X_train, y_train, epochs=100, validation_data=(X_valid, y_valid), callbacks=[keras.callbacks.EarlyStopping(patience=10)])

Epoch 1/100
  1/363 [..............................] - ETA: 53s - loss: 3.5737 - mean_squared_error: 3.5737

C:\Users\Administrator\AppData\Local\Temp\ipykernel_25732\3935065707.py:34: DeprecationWarning: KerasRegressor is deprecated, use Sci-Keras (https://github.com/adriangb/scikeras) instead. See https://www.adriangb.com/scikeras/stable/migration.html for help migrating.
  keras_reg = keras.wrappers.scikit_learn.KerasRegressor(build_model)


363/363 [==============================] - 1s 2ms/step - loss: 0.7247 - mean_squared_error: 0.7247 - val_loss: 1.7396 - val_mean_squared_error: 1.7396
Epoch 2/100
363/363 [==============================] - 1s 2ms/step - loss: 0.4637 - mean_squared_error: 0.4637 - val_loss: 0.3950 - val_mean_squared_error: 0.3950
Epoch 3/100
363/363 [==============================] - 1s 2ms/step - loss: 0.5004 - mean_squared_error: 0.5004 - val_loss: 30.9033 - val_mean_squared_error: 30.9033
Epoch 4/100
363/363 [==============================] - 1s 1ms/step - loss: 0.4258 - mean_squared_error: 0.4258 - val_loss: 4.5618 - val_mean_squared_error: 4.5618
Epoch 5/100
363/363 [==============================] - 1s 2ms/step - loss: 0.5674 - mean_squared_error: 0.5674 - val_loss: 6.8596 - val_mean_squared_error: 6.8596
Epoch 6/100
363/363 [==============================] - 1s 1ms/step - loss: 0.4274 - mean_squared_error: 0.4274 - val_loss: 47.0557 - val_mean_squared_error: 47.0557
Epoch 7/100
363/363 [=========

In [6]:
mse_test = keras_reg.score(X_test, y_test)
X_new = X_test[0:5]
y_pred = keras_reg.predict(X_new)

1/1 [==============================] - 0s 40ms/step


In [7]:
from scipy.stats import reciprocal
from sklearn.model_selection import RandomizedSearchCV
import numpy as np

# 设置随机种子
RANDOM_SEED = 42
np.random.seed(RANDOM_SEED)
param_distribs = {
    "n_hidden": [0, 1, 2, 3],
    "n_neurons": np.arange(1, 100),
    "learning_rate": reciprocal(0.0001, 0.1),
    "activation": ["relu", "tanh"],
    "input_shape": [8]
}

rnd_search_cv = RandomizedSearchCV(keras_reg, param_distribs, n_iter=10, cv=3, verbose=2, random_state=42)
rnd_search_cv.fit(X_train, y_train, epochs=100, validation_data=(X_valid, y_valid), callbacks=[keras.callbacks.EarlyStopping(patience=10)])

Fitting 3 folds for each of 10 candidates, totalling 30 fits
Epoch 1/100
242/242 [==============================] - 1s 2ms/step - loss: 1.2102 - mean_squared_error: 1.2102 - val_loss: 40.6440 - val_mean_squared_error: 40.6440
Epoch 2/100
242/242 [==============================] - 1s 2ms/step - loss: 0.6174 - mean_squared_error: 0.6174 - val_loss: 3.7724 - val_mean_squared_error: 3.7724
Epoch 3/100
242/242 [==============================] - 0s 2ms/step - loss: 0.5102 - mean_squared_error: 0.5102 - val_loss: 0.4559 - val_mean_squared_error: 0.4559
Epoch 4/100
242/242 [==============================] - 0s 2ms/step - loss: 0.3672 - mean_squared_error: 0.3672 - val_loss: 0.3447 - val_mean_squared_error: 0.3447
Epoch 5/100
242/242 [==============================] - 0s 2ms/step - loss: 0.3535 - mean_squared_error: 0.3535 - val_loss: 0.3530 - val_mean_squared_error: 0.3530
Epoch 6/100
242/242 [==============================] - 0s 2ms/step - loss: 0.3479 - mean_squared_error: 0.3479 - val_loss:

KeyboardInterrupt: 

In [ ]:
print(rnd_search_cv.best_params_)
print(rnd_search_cv.best_score_)
model = rnd_search_cv.best_estimator_.model
